In [0]:
import cv2
import numpy as np
!pip install pafy
!pip install youtube_dl 
import pafy

     |████████████████████████████████| 1.8MB 2.6MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#url of the video to predict Age and gender
url = 'https://www.youtube.com/watch?v=Qabkux6Wz5g'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4") 

In [0]:
cap = cv2.VideoCapture(play.url)

In [0]:
cap.set(3, 480) #set width of the frame
cap.set(4, 640) #set height of the frame

False

In [0]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

In [0]:
def load_caffe_models():
  age_net = cv2.dnn.readNetFromCaffe('/content/drive/My Drive/Colab Notebooks/predictor/deploy_age.prototxt', '/content/drive/My Drive/Colab Notebooks/predictor/age_net.caffemodel')
  gender_net = cv2.dnn.readNetFromCaffe('/content/drive/My Drive/Colab Notebooks/predictor/deploy_gender.prototxt', '/content/drive/My Drive/Colab Notebooks/predictor/gender_net.caffemodel')
  return(age_net, gender_net)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def video_detector(age_net, gender_net):
  frame_array = []
  font = cv2.FONT_HERSHEY_SIMPLEX
  while True:
    ret, image = cap.read()
    if not ret:
      make_video(frame_array, fps=30)
      break   
    face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/Colab Notebooks/predictor/haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    if(len(faces)>0):
      print("Found {} faces".format(str(len(faces))))
      for (x, y, w, h )in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)
        #Get Face 
        face_img = image[y:y+h, h:h+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        #Predict Gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        print("Gender : " + gender)
        #Predict Age
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_list[age_preds[0].argmax()]
        print("Age Range: " + age)
        overlay_text = "%s %s" % (gender, age)
        cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        frame_array.append(image)
        #0xFF is a hexadecimal constant which is 11111111 in binary.
        if cv2.waitKey(1) & 0xFF == ord('q'):
          make_video(frame_array, fps=30)
          break
        



In [0]:
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize

def make_video(images=None, fps=5, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
      if vid is None:
        if size is None:
          size = image.shape[1], image.shape[0]
        vid = VideoWriter("/content/drive/My Drive/Colab Notebooks/predictor/out.mp4"
, fourcc, float(fps), size, is_color)
      if size[0] != image.shape[1] and size[1] != image.shape[0]:
            image = resize(image, size)
      vid.write(image)
    vid.release()
    return vid

In [0]:
import glob
import os
if __name__ == "__main__":
  age_net , gender_net = load_caffe_models()
  video_detector(age_net, gender_net)
  

Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (4, 6)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (4, 6)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : M

In [0]:
from google.colab import files
files.download('/content/drive/My Drive/Colab Notebooks/predictor/out.mp4')